Importing dependencies and loading csv file

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ds=pd.read_csv("/content/drive/MyDrive/ML Projects/Twitter_live_Sentiment_Analysis/sentiment140.csv", encoding = "ISO-8859-1")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id"     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
ds.drop(columns=['id"','date','flag','user'],inplace=True)


In [7]:
y=ds['target']

In [8]:
ds.head()


,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
ds.duplicated().sum() #checking for duplicates

16309

In [10]:
ds=ds.drop_duplicates(keep='first')

Target value for all tweets is 0 or 4 at which implies negative and positive

In [11]:
ds = ds.sample(frac = 1)#shuffling

In [12]:
ds.head()

,target,text
133332,0,stomach ache AND homework. bleh.
1238149,4,@cferro I've been known to turn a few heads I...
639274,0,Mac os x installation fail! Unable to set up w...
698543,0,i have a couple of things i wanna bring for th...
359540,0,Looking forward to Iain Lee at 10 tonight. I'v...


In [13]:
ds_split=np.array_split(ds,40)#splitting 1.6 Million tweets into 40 equal parts

In [14]:
ds_split[0]['target'].value_counts()

4    19919
0    19674
Name: target, dtype: int64

In [15]:
df=ds_split[0] #df stores the fraction of our original dataframe
df.head()

,target,text
133332,0,stomach ache AND homework. bleh.
1238149,4,@cferro I've been known to turn a few heads I...
639274,0,Mac os x installation fail! Unable to set up w...
698543,0,i have a couple of things i wanna bring for th...
359540,0,Looking forward to Iain Lee at 10 tonight. I'v...


In [16]:
df=df.reset_index()

In [17]:
df.drop(columns=['index'],inplace=True)

In [18]:
df.head()

,target,text
0,0,stomach ache AND homework. bleh.
1,4,@cferro I've been known to turn a few heads I...
2,0,Mac os x installation fail! Unable to set up w...
3,0,i have a couple of things i wanna bring for th...
4,0,Looking forward to Iain Lee at 10 tonight. I'v...


In [19]:
df['target'].value_counts()

4    19919
0    19674
Name: target, dtype: int64

Preprocessing

In [20]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import nltk

In [22]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [24]:
from nltk.stem.porter import PorterStemmer

In [25]:
import re
import string

In [26]:

def pp(text):

  text=re.sub(r'https?:\/\/.*[\r\n]*','', text)
  text=re.sub(r'#','',text)

  tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True, reduce_len=True)
  text= tokenizer.tokenize(text)

  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text=y[:]
  y.clear()

  stop_words = set(stopwords.words('english'))
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i in text:
    if i not in stop_words and i not in punc:
      y.append(i)

  text=y[:]
  y.clear()

  ps=PorterStemmer()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)


In [27]:
pp(df['text'][1])

'known turn head person compliment rodent pic chang name wanna make'

In [28]:
df['text'][1]

"@cferro I've been known to turn a few heads  I'm the person who complimented you on your rodent pic. I changed my name. So wanna make out?"

In [29]:

df['pp_text']=df[df.columns[1]].apply(pp)
df.loc[df['target'] == 4, 'target'] = 1

In [30]:
df.head()

,target,text,pp_text
0,0,stomach ache AND homework. bleh.,stomach ach homework bleh
1,1,@cferro I've been known to turn a few heads I...,known turn head person compliment rodent pic c...
2,0,Mac os x installation fail! Unable to set up w...,mac os x instal fail unabl set wireless reinst...
3,0,i have a couple of things i wanna bring for th...,coupl thing wanna bring concert tonight dont f...
4,0,Looking forward to Iain Lee at 10 tonight. I'v...,look forward iain lee 10 tonight got noth till


Dropping empty rows

In [31]:
df.dropna(inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39593 entries, 0 to 39592
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   target   39593 non-null  int64 
 1   text     39593 non-null  object
 2   pp_text  39593 non-null  object
dtypes: int64(1), object(2)
memory usage: 928.1+ KB


Model

In [33]:
y=df['target'].values

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=3000)
X=tfidf.fit_transform(df['pp_text']).toarray()
len(tfidf.vocabulary_)

3000

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size =0.3,
                                                    random_state=1)


In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

models = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

results_df = pd.DataFrame(columns=['Accuracy', 'F1 Score', 'Precision', 'Recall'])

for model_name, model in models.items():

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    results_df.loc[model_name] = [accuracy, f1, precision, recall]


In [43]:
print(results_df)

                         Accuracy  F1 Score  Precision    Recall
Multinomial Naive Bayes  0.737414  0.740753   0.731211  0.750547
Logistic Regression      0.740360  0.747916   0.726536  0.770591
Random Forest            0.728658  0.730901   0.724669  0.737241
SVM                      0.742633  0.750020   0.728862  0.772444
K-Nearest Neighbors      0.567267  0.680547   0.539249  0.922183
Decision Tree            0.682691  0.685786   0.678937  0.692774


SVM Classification model gives the best accuracy for our use case

In [42]:
import pickle
pickle.dump(tfidf,open('sentiment_analysis_vectorizer.pkl','wb'))
pickle.dump(SVC(),open('sentiment_analysis_model.pkl','wb'))
